In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import time
import os
from datetime import datetime

# 현재 날짜를 yyyymmdd 형식으로 얻기
current_date = datetime.now().strftime("%Y%m%d")
search_date = datetime.now().strftime("%Y-%m-%d")  # 검색 날짜를 yyyy-mm-dd 형식으로 얻기

# 사용자로부터 저장할 Excel 파일 기본 이름 입력받기
base_file_name = input("저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): ")

# 입력받은 파일 이름에 오늘 날짜 추가
excel_file_name = f"{base_file_name}_{current_date}.xlsx"

# 현재 작업 중인 폴더의 경로를 얻기
current_working_directory = os.getcwd()
file_path = os.path.join(current_working_directory, excel_file_name)  # 파일 경로 결합

# WebDriver 설정(Chrome 사용)
driver = webdriver.Chrome()

# Excel 설정
wb = Workbook()
wb.remove(wb['Sheet'])  # 기본 생성된 시트 제거

# 네이버 데이터랩 쇼핑 인사이트 페이지로 이동
driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")

# 필요한 요소가 로드될 때까지 대기하는 WebDriverWait 인스턴스 생성
wait = WebDriverWait(driver, 20)

# 드롭다운 및 옵션 선택 경로
dropdown_path = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span'

# 옵션 이름 목록
option_names = [
    '패션의류', '패션잡화', '화장품_미용', '디지털_가전', '가구_인테리어', '출산_육아', '식품',
    '스포츠_레저', '생활_건강', '여가_생활편의', '면세점', '도서'
]

# 각 옵션에 대해 순차적으로 처리
for option_num in range(1, 13):
    try:
        # 드롭다운 및 옵션 선택
        dropdown = wait.until(EC.element_to_be_clickable((By.XPATH, dropdown_path)))
        dropdown.click()
        option_xpath = f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[{option_num}]/a'
        option = wait.until(EC.element_to_be_clickable((By.XPATH, option_xpath)))
        option.click()
        time.sleep(1)  # 드롭다운 선택 후 대기

        # 기기별, 성별, 연령 전체 선택
        driver.find_element(By.XPATH, '//*[@id="18_device_0"]').click()
        driver.find_element(By.XPATH, '//*[@id="19_gender_0"]').click()
        age_checkbox = driver.find_element(By.XPATH, '//*[@id="20_age_0"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", age_checkbox)
        age_checkbox.click()

        time.sleep(2)  # 필요한 요소 로딩 대기

        # '조회하기' 버튼 클릭
        search_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/a')))
        search_button.click()

        # 새로운 시트 생성 및 설정
        sheet_name = option_names[option_num - 1]
        ws = wb.create_sheet(sheet_name)
        ws.append(['순위', '인기검색어', '검색 날짜'])

        # 결과 스크래핑 및 Excel 파일 저장 (25페이지)
        for page in range(2):  # 페이지 수 조정 (이 곳을 조정해 주세요)
            time.sleep(2)  # 페이지 로딩 대기
            for j in range(1, 21):  # 한 페이지당 최대 20개 아이템
                try:
                    path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{j}]/a'
                    result = driver.find_element(By.XPATH, path).text
                    result_data = result.split('\n')
                    result_data[0] = int(result_data[0])  # 순위를 숫자 형식으로 변환
                    result_data.append(search_date)  # 검색 날짜 추가
                    print(result_data)
                    ws.append(result_data)
                except Exception as e:
                    print(f"Error occurred while scraping data for option {option_num}, page {page + 1}, item {j}: {e}")
                    continue

            # 다음 페이지로 넘어가는 버튼 클릭 로직 (실제 페이지 구조에 따라 달라질 수 있음)
            try:
                next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]')))
                next_button.click()
            except Exception as e:
                print(f"Error occurred while clicking next button on option {option_num}, page {page + 1}: {e}")
                break
    except Exception as e:
        print(f"Error occurred while processing option {option_num}: {e}")
        continue

# 파일 저장
try:
    wb.save(file_path)
    wb.close()
    print(f"'{excel_file_name}' 파일이 현재 폴더에 저장되었습니다.")
except Exception as e:
    print(f"Error occurred while saving the file: {e}")

# 작업 완료 후, 드라이버 종료
driver.quit()

# 저장된 파일 열기
try:
    os.startfile(file_path)
except Exception as e:
    print(f"Error occurred while opening the file: {e}")


저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): 네이버키워드전체
[1, '원피스', '2024-07-03']
[2, '써스데이아일랜드원피스', '2024-07-03']
[3, '여름원피스', '2024-07-03']
[4, '블라우스', '2024-07-03']
[5, '스투시반팔', '2024-07-03']
[6, '자라원피스', '2024-07-03']
[7, '여름가디건', '2024-07-03']
[8, '남자반바지', '2024-07-03']
[9, '시슬리원피스', '2024-07-03']
[10, '티셔츠', '2024-07-03']
[11, '반바지', '2024-07-03']
[12, '라코스테원피스', '2024-07-03']
[13, '써스데이아일랜드', '2024-07-03']
[14, '듀엘원피스', '2024-07-03']
[15, '롱원피스', '2024-07-03']
[16, '점프수트', '2024-07-03']
[17, '잇미샤원피스', '2024-07-03']
[18, '써스데이아일랜드블라우스', '2024-07-03']
[19, '남자반팔티', '2024-07-03']
[20, '지고트원피스', '2024-07-03']
[21, '바스락원피스', '2024-07-03']
[22, '우비', '2024-07-03']
[23, '린넨셔츠', '2024-07-03']
[24, '나이키반바지', '2024-07-03']
[25, '지컷원피스', '2024-07-03']
[26, '버뮤다팬츠', '2024-07-03']
[27, '쉬폰원피스', '2024-07-03']
[28, '여성원피스', '2024-07-03']
[29, '온앤온원피스', '2024-07-03']
[30, '플라스틱아일랜드원피스', '2024-07-03']
[31, '쉬즈미스원피스', '2024-07-03']
[32, '리스트원피스', '2024-07-03']
[33, '데님원피스', '2024-07-03']
[34, '베네통원피스', '202

[21, '자전거', '2024-07-03']
[22, '비키니', '2024-07-03']
[23, '트레킹화', '2024-07-03']
[24, '풋살화', '2024-07-03']
[25, '바베큐그릴', '2024-07-03']
[26, '로드자전거', '2024-07-03']
[27, '골프장갑', '2024-07-03']
[28, '방수팩', '2024-07-03']
[29, '캠핑테이블', '2024-07-03']
[30, '에어텐트', '2024-07-03']
[31, '텐트', '2024-07-03']
[32, '아이스박스', '2024-07-03']
[33, '보냉백', '2024-07-03']
[34, '넥쿨러', '2024-07-03']
[35, '모노키니', '2024-07-03']
[36, '디스커버리반팔티', '2024-07-03']
[37, '축구화', '2024-07-03']
[38, '실내자전거', '2024-07-03']
[39, '레쉬가드', '2024-07-03']
[40, '등산모자', '2024-07-03']
[1, '샤워기헤드', '2024-07-03']
[2, '비데', '2024-07-03']
[3, '스피드랙', '2024-07-03']
[4, '스탠리텀블러', '2024-07-03']
[5, '텀블러', '2024-07-03']
[6, '욕실화', '2024-07-03']
[7, '안마의자', '2024-07-03']
[8, '차량용방향제', '2024-07-03']
[9, '금고', '2024-07-03']
[10, '멀티탭', '2024-07-03']
[11, '스타벅스텀블러', '2024-07-03']
[12, '철제선반', '2024-07-03']
[13, '모기장', '2024-07-03']
[14, 'a4용지', '2024-07-03']
[15, '종아리마사지기', '2024-07-03']
[16, '어깨마사지기', '2024-07-03']
[17, '박스테이프', '2024-07-03']
[18,